In [10]:
####################################################################################
# This test code shows how new_basin_cities looks like
# basin at each city
distance=100
min_of_cities=0
max_of_basinnum=1000
target_city = 14
####################################################################################
h08dir = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08'
basin_cities_path = f'{h08dir}/global_city/dat/riv_num_/basin_to_cities_{distance}km.json'

with open(basin_cities_path, 'r') as json_file:
    tmp = json.load(json_file)
    
sorted_keys = sorted(tmp.keys(), key=lambda k: float(k))

target_key_list = []

for key in sorted_keys:
    value = tmp[key]
    if min_of_cities < len(value) and float(key) < max_of_basinnum:
        for i in value:
            if int(float(i)) == target_city:
                target_key_list.append(key)
        #print(f'key{key}, len: {len(value)}')
        #print(value)
        #print('-----------------------------------------')
print(f'target_city: {target_city}')
print(f'target_key_list: {target_key_list}')

target_city: 14
target_key_list: ['23.0']


In [11]:
target_key = '23.0'
value = tmp[target_key]
print(f'target_key: {target_key}')
print(f'value: {value}')

target_key: 23.0
value: ['14', '33', '111', '146', '221', '260', '272', '276', '284', '296', '354', '370', '466', '536', '578', '715', '764', '813', '849', '1091', '1103', '1196', '1220', '1295', '1405', '1430', '1519', '1535', '1536', '1563', '1582', '1636', '1689']


In [12]:
keys_lst = list(tmp.keys())
key_index = keys_lst.index(target_key)
print(f'rivnum_key_index: {key_index}')

rivnum_key_index: 15


In [13]:
import pandas as pd
mcy_pop_water_path = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08/global_city/dat/cty_lst_/gpw4/mcy_pop_water_12region.txt'
column_names = ['index', 'flag', 'rate', 'wup', 'pop', 'ava', 'mpcy', 'potential', 'region', 'country', 'cityname', '12region']
df = pd.read_csv(mcy_pop_water_path, delimiter='|', header=None, names=column_names)
df_filtered_test = df[df['index'] == target_city]
display(df_filtered_test.head())

,index,flag,rate,wup,pop,ava,mpcy,potential,region,country,cityname,12region
13,14,False,0.92072,12611.924,11612047.0,4.488725e+08,38.655756,37.0,southern asia,Pakistan,Karachi,South Asia
